# Assessment 4 


# Importing HPC Files

Here we just import all of our data sets and we later combine them into one suitable data frame for our Neural Network and then perform some pre-processing of the data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(10)
import pickle


df_1 = pd.read_csv("https://github.com/Galeforse/DST-Assessment-04/raw/main/Data/UNS1.zip",header=None)
df_2 = pd.read_csv("https://github.com/Galeforse/DST-Assessment-04/raw/main/Data/UNS2.zip",header=None)
df_3 = pd.read_csv("https://github.com/Galeforse/DST-Assessment-04/raw/main/Data/UNS3.zip",header=None)
df_4 = pd.read_csv("https://github.com/Galeforse/DST-Assessment-04/raw/main/Data/UNS4.zip",header=None)

C:\Users\corri\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,3,47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\corri\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,39,47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


At this point we saw that the header names are not exactly easy to read so we just slightly adjust them from the original headers.

In [2]:
colnames = (['source_ip', 'source_port', 'dest_ip', 'dest_port', 'proto', 'state', 'duration', 'source_bytes', 'dest_bytes', 'source_ttl',
             'dest_ttl', 'source_loss', 'dest_loss', 'service', 'source_load', 'dest_load', 'source_pkts', 'dest_pkts', 'source_TP_win', 'dest_TP_win', 
             'source_tcp_bn', 'dest_tcp_bn', 'source_mean_sz', 'dest_mean_sz', 'trans_depth', 'res_bdy_len', 'source_jitter', 'dest_jitter', 'start_time',
             'last_time', 'source_int_pk_time', 'dest_int_pk_time', 'tcp_rtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'count_state_ttl', 
             'count_flw_http_mthd', 'is_ftp_login', 'count_ftp_cmd', 'count_srv_source', 'count_srv_dest', 'count_dest_ltm',
             'count_source_ltm', 'count_source_destport_ltm', 'count_dest_sourceport_ltm', 'counts_dest_source_ltm', 'attack_cat', 'Label'])
df_1.columns = colnames
df_2.columns = colnames
df_3.columns = colnames
df_4.columns = colnames

In [3]:
data_frames = [df_1,df_2,df_3,df_4]
data = pd.concat(data_frames)

# EDA 

In this section, the aim is to get our data into the same format as if we were running a neural network on it. This will involve checking for inf and NaN values, any missingnss in our data and ensuring our data is of the correct type to be classified, namely integer or float.

In [4]:
data.head()

,source_ip,source_port,dest_ip,dest_port,proto,state,duration,source_bytes,dest_bytes,source_ttl,...,count_ftp_cmd,count_srv_source,count_srv_dest,count_dest_ltm,count_source_ltm,count_source_destport_ltm,count_dest_sourceport_ltm,counts_dest_source_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0


In [5]:
#DataFrame with columns
columns = pd.DataFrame(list(data.columns.values[1:]))

#DataFrame with data types
data_types = pd.DataFrame(data.dtypes, columns=['Data Type'])

#DataFrame with unique values
unique_value_counts = pd.DataFrame(columns=['Unique Values'])
for v in list(data.columns.values):
    unique_value_counts.loc[v] = [data[v].nunique()]

missing_data_counts = pd.DataFrame(data.isnull().sum(), columns=['Missing Values'])
data_quality_report = data_types.join(unique_value_counts).join(missing_data_counts)
print('Data Quality Report')
data_quality_report

Data Quality Report


,Data Type,Unique Values,Missing Values
source_ip,object,43,0
source_port,object,100341,0
dest_ip,object,47,0
dest_port,object,128297,0
proto,object,135,0
state,object,16,0
duration,float64,587303,0
source_bytes,int64,14155,0
dest_bytes,int64,19166,0
source_ttl,int64,13,0


From this we can see that there are a number of columns of the wrong data type and several with missing values. Attack_cat is expected to have NaN values as this was touched upon into the introduction for the data. From Alex's analysis we know that count_flw_http_mthd and is_ftp_login are missing values and have NaN values which we know how to fix. 

Let us first try and replace some missing values.

From the description for is_ftp_login we see that the values should only be 0 or 1 but we have 2 and 4 and we are not quite sure why but since its median is 0 we will jsut impute the data as 0. Similarily for Count_flw_http_mthd, its median is also 0 so we also impute missing values as 0.

In [6]:
data =data.fillna(0)

In [7]:
data.groupby('count_flw_http_mthd').size()

count_flw_http_mthd
0.0     2334936
1.0      188008
2.0         588
3.0         738
4.0        6564
5.0         430
6.0        7902
8.0          56
9.0         270
10.0         30
12.0        132
14.0        224
16.0         48
25.0         25
30.0         60
36.0         36
dtype: int64

In [8]:
data.groupby('is_ftp_login').size()

is_ftp_login
0.0    2496472
1.0      43389
2.0         30
4.0        156
dtype: int64

Lets see how count_ftp_cmd is made up.

In [9]:
data.groupby('count_ftp_cmd').size()


count_ftp_cmd
0    1056339
1      17087
2       1234
3        729
4        804
5        290
6        332
8         18
     1429879
0      10159
1      22990
2         30
4        156
dtype: int64

As there are just missing values we simply replace them with 0.

In [10]:
data['count_ftp_cmd'] = data['count_ftp_cmd'].replace(' ',0)
data.groupby('count_ftp_cmd').size()


count_ftp_cmd
0    2486218
1      17087
2       1234
3        729
4        804
5        290
6        332
8         18
0      10159
1      22990
2         30
4        156
dtype: int64

Now we must replace all the object data types to either float or integer types. I tried using to.numeric from pandas but due to the values this proved more hassle than it was worth, especially given Matt has a tried and tested method which involes mapping the data to new columns and removing the old columns as follows.

In [11]:
data_source_ip = pd.DataFrame(data['source_ip'])
data_source_port = pd.DataFrame(data['source_port'])
data_dest_ip = pd.DataFrame(data['dest_ip'])
data_dest_port = pd.DataFrame(data['dest_port'])
data_proto = pd.DataFrame(data['proto'])
data_state = pd.DataFrame(data['state'])
data_service = pd.DataFrame(data['service'])
data_count_ftp_cmd = pd.DataFrame(data['count_ftp_cmd'])
data_attack_cat = pd.DataFrame(data['attack_cat'])

In [12]:
sips = data.source_ip.unique()
sip_dict = dict(zip(sips,range(len(sips))))

sp = data.source_port.unique()
sp_dict = dict(zip(sp,range(len(sp))))
               
dips = data.dest_ip.unique()
dip_dict = dict(zip(dips,range(len(dips))))

dp = data.dest_port.unique()
dp_dict = dict(zip(dp,range(len(dp))))

p = data.proto.unique()
p_dict = dict(zip(p,range(len(p))))

states = data.state.unique()
state_dict = dict(zip(states,range(len(states))))

services = data.service.unique()
service_dict = dict(zip(services,range(len(services))))

cfc = data.count_ftp_cmd.unique()
cfc_dict = dict(zip(cfc,range(len(cfc))))

ac = data.attack_cat.unique()
ac_dict = dict(zip(ac,range(len(ac))))

In [13]:
#Mapping the dictionaries to data frames 
data['source_ip_int'] = data['source_ip'].map(sip_dict)
data['source_port_int'] = data['source_port'].map(sp_dict)
data['dest_ip_int'] = data['dest_ip'].map(dip_dict)
data['dest_port_int'] = data['dest_port'].map(dp_dict)
data['proto_int'] = data['proto'].map(p_dict)
data['state_int'] = data['state'].map(state_dict)
data['service_int'] = data['service'].map(service_dict)
data['count_ftp_cmd_int'] = data['count_ftp_cmd'].map(cfc_dict)
data['attack_cat_int'] = data['attack_cat'].map(ac_dict)

In [14]:
#Reomving the old data frames
data = data.drop('source_ip',axis=1)
data = data.drop('source_port',axis=1)
data = data.drop('dest_ip',axis=1)
data = data.drop('dest_port',axis=1)
data = data.drop('proto',axis=1)
data = data.drop('state',axis=1)
data = data.drop('service',axis=1)
data = data.drop('count_ftp_cmd',axis=1)
data = data.drop('attack_cat',axis=1)

In [15]:
#DataFrame with columns
columns = pd.DataFrame(list(data.columns.values[1:]))

#DataFrame with data types
data_types = pd.DataFrame(data.dtypes, columns=['Data Type'])

#DataFrame with unique values
unique_value_counts = pd.DataFrame(columns=['Unique Values'])
for v in list(data.columns.values):
    unique_value_counts.loc[v] = [data[v].nunique()]

missing_data_counts = pd.DataFrame(data.isnull().sum(), columns=['Missing Values'])
data_quality_report = data_types.join(unique_value_counts).join(missing_data_counts)
print('Data Quality Report')
data_quality_report

Data Quality Report


,Data Type,Unique Values,Missing Values
duration,float64,587303,0
source_bytes,int64,14155,0
dest_bytes,int64,19166,0
source_ttl,int64,13,0
dest_ttl,int64,11,0
source_loss,int64,544,0
dest_loss,int64,707,0
source_load,float64,1100258,0
dest_load,float64,1173739,0
source_pkts,int64,933,0


As we can see from the data report, we have removed all of the missing values and all of the data types are now correct, allowing us to move onto the scaling stage for our data.

# Scaling

We aim to get our data to be as similar as possible to the data we would feed into a Neural Network to ensure a direct comparison between our Naive Bayes (NB) classification model and other team members Neural Networks which will allow us to directly assess which method is optimal for such a problem.  

Obviously for our classification problem we need some classes. Choosing Normal and Non-Normal traffic would be intutitive and we can draw upon similar things we used from Assessment 1. However, we have many types of attacks now but fortunaetly Naive Bayes is equipped to handle this. We can see all our categories by calling ac_dict which was generated during the re-mapping of columns so we have all we need to feed this through the classifier and we can still use alot from what we learned in Assessment 1.

For Neural Networks, certain data scaling is imposed as unscaled input variables can result in a slow learning process and sometimes causing the learning to fail completely and what to scale the data to depends on the activation function being used in the case. As we mentioned before we will be manipulating our data to be the same for Naive Bayes and Neural Networks. Luckily for us, this scaling will have minimal impact on NB since NB sets the priors based on the data we feed it and scale them to match our data.

In [16]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [17]:
scaler = preprocessing.StandardScaler().fit(data)
data_scaled = scaler.transform(data)
data_scaled.shape

(2540047, 49)

# Classifier - Naive Bayes

After our data is scaled we spilt it into training and test data sets as before in Assessment 1 and we would do the same for our Neural Network to see how well the model is able to learn and classify data.

In [18]:
from sklearn.model_selection import train_test_split
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
Y = data['attack_cat_int']
X_train, X_test, Y_train, Y_test = train_test_split(data_scaled, Y, test_size = 0.1, random_state = 10)

In [29]:
Y_train

180347    0
286963    0
444767    4
625160    0
333935    0
         ..
104335    0
648316    0
576279    0
529741    0
348547    0
Name: attack_cat_int, Length: 2286042, dtype: int64

In [20]:
def model(model_name, X_train, Y_train, X_test, Y_test):
    '''
     Fits the model on train data and predict the performance on train and test data.
     '''
    print('Fitting the model and prediction on train data:')
    start = dt.datetime.now()
    model_name.fit(X_train, Y_train)
    y_tr_pred = model_name.predict(X_train)
    print('Completed')
    print('Time taken:',dt.datetime.now()-start)
    print('='*50)
    
    results_tr = dict()
    y_tr_pred = model_name.predict(X_train)    
    results_tr['precision'] = precision_score(Y_train, y_tr_pred,         average='weighted')
    results_tr['recall'] = recall_score(Y_train, y_tr_pred, average='weighted')
    results_tr['f1_score'] = f1_score(Y_train, y_tr_pred, average='weighted')
    
    results_test = dict()
    print('Prediction on test data:')
    start = dt.datetime.now()
    y_test_pred = model_name.predict(X_test)
    print('Completed')
    print('Time taken:',dt.datetime.now()-start)
    print('='*50)
    
    print('Performance metrics:')
    print('='*50)
    print('Confusion Matrix is:')
    confusion_matrix_func(Y_test, y_test_pred)
    print('='*50)
    results_test['precision'] = precision_score(Y_test, y_test_pred, average='weighted')
    print('Precision score is:')
    print(precision_score(Y_test, y_test_pred, average='weighted'))
    print('='*50)
    results_test['recall'] = recall_score(Y_test, y_test_pred, average='weighted')
    print('Recall score is:')
    print(recall_score(Y_test, y_test_pred, average='weighted'))
    print('='*50)
    results_test['f1_score'] = f1_score(Y_test, y_test_pred, average='weighted')
    print('F1-score is:')
    print(f1_score(Y_test, y_test_pred, average='weighted'))
    # add the trained  model to the results
    results_test['model'] = model
    
    return results_tr, results_test, y_test_pred

In [21]:
def confusion_matrix_func(Y_test, y_test_pred):
    
    C = confusion_matrix(Y_test, y_test_pred)
    cm_df = pd.DataFrame(C)
    labels = ['Exploits', 'Reconnaissance', 'DoS', 'Generic', 'Shellcode', ' Fuzzers', 'Worms', 'Backdoors', 'Analysis']
    plt.figure(figsize=(30,25))
    sns.set(font_scale=1.4)
    sns.heatmap(cm_df, annot=True, annot_kws={"size":12}, fmt='g',       xticklabels=labels, yticklabels=labels)
    plt.ylabel('Actual Class')
    plt.xlabel('Predicted Class')
    
    plt.show()

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

In [ ]:
hyperparameter = {'var_smoothing':[10**x for x in range(-9,3)]}
nb = GaussianNB()
nb_grid = GridSearchCV(nb, param_grid=hyperparameter, cv=5, verbose=1, n_jobs=-1)
nb_results_train, nb_results_test, Y_pred = model(nb, X_train, Y_train, X_test, Y_test)

Fitting the model and prediction on train data:
Completed
Time taken: 0:00:15.607183
